# Modeling and predicting `SalePrice`

# Load and inspect data

In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from preprocess import data_plus, hp_df_plus, description

In [2]:
# load preprocessed data
hp_data = data_plus({df_name: hp_df_plus(pd.read_csv(df_name + '.csv',
                                         index_col=['split', 'Id']))
                     for df_name in ['orig', 'edit', 'model']})
orig, edit, model = (hp_data.dfs['orig'], hp_data.dfs['edit'],
                     hp_data.dfs['model'])

model.head()

OverallQual  OverallCond  ExterQual  ExterCond  BsmtQual  BsmtCond  \
split Id                                                                       
train 1      0.657759    -0.517236   1.060445  -0.238371  0.622962  0.118007   
      2     -0.069369     2.177448  -0.689349  -0.238371  0.622962  0.118007   
      3      0.657759    -0.517236   1.060445  -0.238371  0.622962  0.118007   
      4      0.657759    -0.517236  -0.689349  -0.238371 -0.653628  1.927866   
      5      1.384886    -0.517236   1.060445  -0.238371  0.622962  0.118007   

          BsmtExposure  BsmtFinType1  HeatingQC  FullBath  ...  \
split Id                                                   ...   
train 1      -0.588375      1.166364   0.892827  0.792856  ...   
      2       2.234662      0.691806   0.892827  0.792856  ...   
      3       0.352637      1.166364   0.892827  0.792856  ...   
      4      -0.588375      0.691806  -0.149520 -1.026343  ...   
      5       1.293650      1.166364   0.892827  0.792856  ...   

          GarageType_Basment  GarageType_BuiltIn  GarageType_CarPort  \
split Id                                                               
train 1            -0.101991           -0.172285           -0.069481   
      2            -0.101991           -0.172285           -0.069481   
      3            -0.101991           -0.172285           -0.069481   
      4            -0.101991           -0.172285           -0.069481   
      5            -0.101991           -0.172285           -0.069481   

          GarageType_Detchd  SaleCondition_Abnorml  SaleCondition_AdjLand  \
split Id                                                                    
train 1           -0.478016              -0.272917              -0.052468   
      2           -0.478016              -0.272917              -0.052468   
      3           -0.478016              -0.272917              -0.052468   
      4           -0.478016               3.664116              -0.052468   
      5           -0.478016              -0.272917              -0.052468   

          SaleCondition_Alloca  SaleCondition_Family  SaleCondition_Normal  \
split Id                                                                     
train 1              -0.091129             -0.117974              0.466058   
      2              -0.091129             -0.117974              0.466058   
      3              -0.091129             -0.117974              0.466058   
      4              -0.091129             -0.117974             -2.145658   
      5              -0.091129             -0.117974              0.466058   

          SaleCondition_Partial  
split Id                         
train 1               -0.303651  
      2               -0.303651  
      3               -0.303651  
      4               -0.303651  
      5               -0.303651  

[5 rows x 118 columns]

In [3]:
model.info()

<class 'preprocess.hp_df_plus'>
MultiIndex: 2916 entries, (train, 1) to (test, 2919)
Columns: 118 entries, OverallQual to SaleCondition_Partial
dtypes: float64(118)
memory usage: 2.7+ MB


# Individual Models

In [4]:
# train and test split
X, y = model.drop(columns=['log_SalePrice']), model['log_SalePrice']
X_train, X_test, y_train = X.loc['train', :], X.loc['test', :], y.loc['train', :], 

In [5]:
X_train.head()

,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,FullBath,...,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0.657759,-0.517236,1.060445,-0.238371,0.622962,0.118007,-0.588375,1.166364,0.892827,0.792856,...,-0.101991,-0.172285,-0.069481,-0.478016,-0.272917,-0.052468,-0.091129,-0.117974,0.466058,-0.303651
2,-0.069369,2.177448,-0.689349,-0.238371,0.622962,0.118007,2.234662,0.691806,0.892827,0.792856,...,-0.101991,-0.172285,-0.069481,-0.478016,-0.272917,-0.052468,-0.091129,-0.117974,0.466058,-0.303651
3,0.657759,-0.517236,1.060445,-0.238371,0.622962,0.118007,0.352637,1.166364,0.892827,0.792856,...,-0.101991,-0.172285,-0.069481,-0.478016,-0.272917,-0.052468,-0.091129,-0.117974,0.466058,-0.303651
4,0.657759,-0.517236,-0.689349,-0.238371,-0.653628,1.927866,-0.588375,0.691806,-0.149520,-1.026343,...,-0.101991,-0.172285,-0.069481,-0.478016,3.664116,-0.052468,-0.091129,-0.117974,-2.145658,-0.303651
5,1.384886,-0.517236,1.060445,-0.238371,0.622962,0.118007,1.293650,1.166364,0.892827,0.792856,...,-0.101991,-0.172285,-0.069481,-0.478016,-0.272917,-0.052468,-0.091129,-0.117974,0.466058,-0.303651


In [6]:
X_test.head()

,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,FullBath,...,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1461,-0.796497,0.380992,-0.689349,-0.238371,-0.653628,0.118007,-0.588375,-0.257310,-1.191866,-1.026343,...,-0.101991,-0.172285,-0.069481,-0.478016,-0.272917,-0.052468,-0.091129,-0.117974,0.466058,-0.303651
1462,-0.069369,0.380992,-0.689349,-0.238371,-0.653628,0.118007,-0.588375,0.691806,-1.191866,-1.026343,...,-0.101991,-0.172285,-0.069481,-0.478016,-0.272917,-0.052468,-0.091129,-0.117974,0.466058,-0.303651
1463,-0.796497,-0.517236,-0.689349,-0.238371,0.622962,0.118007,-0.588375,1.166364,-0.149520,0.792856,...,-0.101991,-0.172285,-0.069481,-0.478016,-0.272917,-0.052468,-0.091129,-0.117974,0.466058,-0.303651
1464,-0.069369,0.380992,-0.689349,-0.238371,-0.653628,0.118007,-0.588375,1.166364,0.892827,0.792856,...,-0.101991,-0.172285,-0.069481,-0.478016,-0.272917,-0.052468,-0.091129,-0.117974,0.466058,-0.303651
1465,1.384886,-0.517236,1.060445,-0.238371,0.622962,0.118007,-0.588375,0.691806,0.892827,0.792856,...,-0.101991,-0.172285,-0.069481,-0.478016,-0.272917,-0.052468,-0.091129,-0.117974,0.466058,-0.303651


## `sklearn` models

In [8]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, BayesianRidge
from sklearn.cross_decomposition import PLSRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score

)

In [9]:
sklearn_models_df = pd.DataFrame(columns=['train_rmse', 'cv_rmse'], index=models.keys())

kf = KFold(n_splits=10, shuffle=True, random_state=27)

for model in models: 
    train_rmse = np.sqrt(mean_squared_error(models[model].predict(X_train), y_train))
    cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=models[model], 
                                                  X=X_train, 
                                                  y=y_train,
                                                  cv=kf,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))
    sklearn_models_df.loc[model, 'train_rmse'] = train_rmse
    sklearn_models_df.loc[model, 'cv_rmse'] = cv_rmse
    
sklearn_models_df

,train_rmse,cv_rmse
lasso,0.399555,0.399685
ridge,0.104347,0.115097
bayes_ridge,0.104781,0.114942
plsreg,0.127295,0.131832
svr,0.0744879,0.149396
knn,0.146542,0.185313
mlp,0.19272,1.41366
dec_tree,0.00103961,0.189558
extra_tree,0.00103939,0.212456


## Ridge regressor

### Hyperparameter defaults for baseline

In [10]:
ridge_default = Ridge()
%timeit -n1 -r1 ridge_default.fit(X_train, y_train)

6.32 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
# R^2 of model
ridge_default.score(X_train, y_train)

0.9317968947876267

In [31]:
ridge_default.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [20]:
ridge_default_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=ridge_default, 
                                                         X=X_train, 
                                                         y=y_train,
                                                         cv=kf,
                                                         scoring='neg_mean_squared_error',
                                                         n_jobs=-1)))

ridge_default_train_rmse = np.sqrt(mean_squared_error(ridge_default.predict(X_train),
                                               y_train))

reg_results_df = pd.DataFrame(columns=['train_rmse', 'cv_rmse'])
reg_results_df.loc['ridge_default', 'train_rmse'] = ridge_default_train_rmse
reg_results_df.loc['ridge_default', 'cv_rmse'] = ridge_default_cv_rmse
reg_results_df

,train_rmse,cv_rmse
ridge_default,0.104347,0.115097


### Tuning with `hyperopt`

In [21]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
from functools import partial

ridge_space = {'alpha': hp.uniform('alpha', low=-3*np.log(10), high=2*np.log(10))}


def ho_cv_rmse(search_params, X=X, y=y, fixed_params={}, est_name='bayes_ridge'):
    if est_name == 'ridge':
        est = Ridge(**{**search_params, **fixed_params})
    if est_name == 'bayes_ridge':
        est = BayesianRidge(**{**search_params, **fixed_params})
    return np.sqrt(-np.mean(cross_val_score(estimator=est, 
                                                  X=X, 
                                                  y=y,
                                                  cv=kf,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

In [22]:
n = 5
ridge_bests = {}

for i in range(n):
    ridge_bests[i] =

100%|██████████| 30/30 [00:04<00:00,  8.45it/s, best loss: 0.11502728066328158]


In [32]:
ridge_ho = Ridge(**ridge_bests[2])
ridge_ho.get_params()

{'alpha': 4.599743103330274,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [23]:
ridge_ho_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=ridge_ho, 
                                                         X=X_train, 
                                                         y=y_train,
                                                         cv=kf,
                                                         scoring='neg_mean_squared_error',
                                                         n_jobs=-1)))
ridge_ho.fit(X_train, y_train)
ridge_ho_train_rmse = np.sqrt(mean_squared_error(ridge_ho.predict(X_train),
                                               y_train))
reg_results_df.loc['ridge_ho', 'train_rmse'] = ridge_ho_train_rmse
reg_results_df.loc['ridge_ho', 'cv_rmse'] = ridge_ho_cv_rmse
reg_results_df

,train_rmse,cv_rmse
ridge_default,0.104347,0.115097
ridge_ho,0.104362,0.115027


## SVR

### Linear and rbf defaults for baseline

In [81]:
svr_linear_default = SVR(kernel='linear')
svr_rbf_default = SVR(kernel='rbf')

In [82]:
%%timeit -n1 -r1 svr_linear_default.fit(X_train, y_train)
svr_rbf_default.fit(X_train, y_train)

167 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [83]:
svr_linear_default.score(X_train, y_train)

0.9289393830305982

In [84]:
svr_rbf_default.score(X_train, y_train)

0.9652448852452377

In [85]:
svr_linear_default.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'auto_deprecated',
 'kernel': 'linear',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [86]:
svr_rbf_default.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'auto_deprecated',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [88]:
svr_lin_def_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=svr_linear_default, 
                                                         X=X_train, 
                                                         y=y_train,
                                                         cv=kf,
                                                         scoring='neg_mean_squared_error',
                                                         n_jobs=-1)))

svr_lin_def_train_rmse = np.sqrt(mean_squared_error(svr_linear_default.predict(X_train),
                                               y_train))

svr_rbf_def_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=svr_rbf_default, 
                                                         X=X_train, 
                                                         y=y_train,
                                                         cv=kf,
                                                         scoring='neg_mean_squared_error',
                                                         n_jobs=-1)))

svr_rbf_def_train_rmse = np.sqrt(mean_squared_error(svr_rbf_default.predict(X_train),
                                               y_train))

reg_results_df.loc['svr_linear_default', 'train_rmse'] = svr_lin_def_train_rmse
reg_results_df.loc['svr_linear_default', 'cv_rmse'] = svr_lin_def_cv_rmse
reg_results_df.loc['svr_rbf_default', 'train_rmse'] = svr_rbf_def_train_rmse
reg_results_df.loc['svr_rbf_default', 'cv_rmse'] = svr_rbf_def_cv_rmse

reg_results_df

,train_rmse,cv_rmse
ridge_default,0.104347,0.115097
ridge_ho,0.104362,0.115027
svr_default,0.10651,0.115355
svr_linear_default,0.10651,0.115355
svr_rbf_default,0.0744879,0.149396


### Tuning with `hyperopt`

In [106]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
from functools import partial

svr_linear_space = {'kernel': 'linear',
                    'C': hp.loguniform('C', low=-3*np.log(10), high=2*np.log(10))
                    }
svr_poly_space = {'kernel': 'poly',
                  'degree': hp.choice('degree', [2, 3]),
                  'coef0': 1,
                  'C': hp.loguniform('C', low=-3*np.log(10), high=2*np.log(10)),
                  'gamma': hp.loguniform('gamma', low=-3*np.log(10), high=1*np.log(10))
                    }
svr_rbf_space = {'kernel': 'rbf',
                 'C': hp.loguniform('C', low=-3*np.log(10), high=2*np.log(10)),
                 'gamma': hp.loguniform('gamma', low=-3*np.log(10), high=1*np.log(10))
                 }

In [116]:
def ho_cv_rmse(search_params, X=X, y=y, fixed_params={}, est_name='bayes_ridge'):
    params = {**search_params, **fixed_params}
    if est_name == 'ridge':
        est = Ridge(**params)
    if est_name == 'bayes_ridge':
        est = BayesianRidge(**params)
    if est_name == 'svr':
        est = SVR(**params)
    return np.sqrt(-np.mean(cross_val_score(estimator=est, 
                                                  X=X, 
                                                  y=y,
                                                  cv=kf,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

In [121]:
svr_linear_bests = {}

for i in range(5):
    svr_bests[i] = fmin(fn=partial(ho_cv_rmse, X=X_train, y=y_train, fixed_params={}, est_name='svr'),
                    space=svr_linear_space,
                    algo=tpe.suggest,
                    max_evals=10)

 80%|████████  | 8/10 [1:44:50<08:33, 256.94s/it, best loss: 0.11543028069127077]   


KeyboardInterrupt: 

In [120]:
svr_poly_bests = {}

for i in range(20):
    svr_poly_bests[i] = fmin(fn=partial(ho_cv_rmse, X=X_train, y=y_train, fixed_params={}, est_name='svr'),
                    space=svr_poly_space,
                    algo=tpe.suggest,
                    max_evals=15)

100%|██████████| 15/15 [00:15<00:00,  1.07s/it, best loss: 0.12362209851379824]


In [97]:
svr_ho = SVR(**svr_bests[7])

svr_ho_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=svr_ho, 
                                                         X=X_train, 
                                                         y=y_train,
                                                         cv=10,
                                                         scoring='neg_mean_squared_error',
                                                         n_jobs=-1)))
svr_ho.fit(X_train, y_train)
svr_ho_train_rmse = np.sqrt(mean_squared_error(svr_ho.predict(X_train),
                                               y_train))
reg_results_df.loc['svr_ho', 'train_rmse'] = svr_ho_train_rmse
reg_results_df.loc['svr_ho', 'cv_rmse'] = svr_ho_cv_rmse
reg_results_df

TypeError: __init__() got an unexpected keyword argument 'C_poly'

In [26]:
svr_default_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=svr_default, 
                                                         X=X_train, 
                                                         y=y_train,
                                                         cv=kf,
                                                         scoring='neg_mean_squared_error',
                                                         n_jobs=-1)))

svr_default_train_rmse = np.sqrt(mean_squared_error(svr_default.predict(X_train),
                                               y_train))


In [28]:
reg_results_df.loc['svr_default', 'train_rmse'] = svr_default_train_rmse
reg_results_df.loc['svr_default', 'cv_rmse'] = svr_default_cv_rmse
reg_results_df

,train_rmse,cv_rmse
ridge_default,0.104347,0.115097
ridge_ho,0.104362,0.115027
svr_default,0.0744879,0.149396


### Tuning `alpha` with `GridSearchCV`

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'alpha': np.linspace(0, 2, 100)}
ridge_search = GridSearchCV(estimator=Ridge(),
                            param_grid=params,
                            scoring='neg_mean_squared_error',
                            n_jobs=-1)
%timeit -n1 -r1 ridge_search.fit(X_train, y_train)

In [ ]:
ridge_search.best_params_

In [ ]:
ridge_grid = Ridge(**ridge_search.best_params_)
ridge_grid.fit(X_train, y_train)
ridge_grid_train_rmse = np.sqrt(mean_squared_error(ridge_grid.predict(X_train),
                                               y_train))
ridge_grid_cv_rmse = np.sqrt(-ridge_search.best_score_)
reg_results_df.loc['ridge_grid', 'train_rmse'] = ridge_grid_train_rmse
reg_results_df.loc['ridge_grid', 'cv_rmse'] = ridge_grid_cv_rmse
reg_results_df

## Bayesian Ridge regressor

### Hyperparameter defaults for baseline

In [ ]:
bayes_ridge_default = BayesianRidge()
%timeit -n1 -r1 bayes_ridge_default.fit(X_train, y_train)

## SVR

In [ ]:
# R^2 of model
bayes_ridge_default.score(X_train, y_train)

In [ ]:
bayes_ridge_default_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=bayes_ridge_default, 
                                                         X=X_train, 
                                                         y=y_train,
                                                         cv=10,
                                                         scoring='neg_mean_squared_error',
                                                         n_jobs=-1)))

bayes_ridge_default_train_rmse = np.sqrt(mean_squared_error(bayes_ridge_default.predict(X_train),
                                               y_train))
reg_results_df.loc['bayes_ridge_default', 'train_rmse'] = bayes_ridge_default_train_rmse
reg_results_df.loc['bayes_ridge_default', 'cv_rmse'] = bayes_ridge_default_cv_rmse
reg_results_df

### Tuning with `hyperopt`

In [ ]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
from functools import partial

bayes_ridge_space = {'alpha_1': hp.loguniform('alpha_1', low=-9*np.log(10), high=3*np.log(10)),
                     'alpha_2': hp.loguniform('alpha_2', low=-9*np.log(10), high=3*np.log(10)),
                     'lambda_1': hp.loguniform('lambda_1', low=-9*np.log(10), high=3*np.log(10)),
                     'lambda_2': hp.loguniform('lambda_2', low=-9*np.log(10), high=3*np.log(10))}


def ho_cv_rmse(search_params, X=X, y=y, fixed_params={}, est_name='bayes_ridge'):
    if est_name == 'bayes_ridge':
        est = BayesianRidge(**{**search_params, **fixed_params})
    return np.sqrt(-np.mean(cross_val_score(estimator=est, 
                                                  X=X, 
                                                  y=y,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

In [ ]:
n = 5
bayes_ridge_bests = {}

for i in range(n):
    bayes_ridge_bests[i] = fmin(fn=partial(ho_cv_rmse, X=X_train, y=y_train, fixed_params={}, est_name='bayes_ridge'),
                    space=bayes_ridge_space,
                    algo=tpe.suggest,
                    max_evals=30)

In [ ]:
bayes_ridge_ho = BayesianRidge(**bayes_ridge_bests[1])

bayes_ridge_ho_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=bayes_ridge_ho, 
                                                         X=X_train, 
                                                         y=y_train,
                                                         cv=10,
                                                         scoring='neg_mean_squared_error',
                                                         n_jobs=-1)))
bayes_ridge_ho.fit(X_train, y_train)
bayes_ridge_ho_train_rmse = np.sqrt(mean_squared_error(bayes_ridge_ho.predict(X_train),
                                               y_train))
reg_results_df.loc['bayes_ridge_ho', 'train_rmse'] = bayes_ridge_ho_train_rmse
reg_results_df.loc['bayes_ridge_ho', 'cv_rmse'] = bayes_ridge_ho_cv_rmse
reg_results_df

##  `XGBRegressor`

### Hyperparameter defaults for baseline

In [ ]:
from xgboost import XGBRegressor

xgbreg_default = XGBRegressor(objective='reg:squarederror', verbosity=1, n_jobs=-1, random_state=27)
%timeit -n1 -r1 xgbreg_default.fit(X_train.values, y_train.values)

In [ ]:
# R^2 of model
xgbreg_default.score(X_train.values, y_train.values)

In [ ]:
xgbreg_default_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=xgbreg_default, 
                                                  X=X_train.values, 
                                                  y=y_train.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

xgbreg_default_train_rmse = np.sqrt(mean_squared_error(xgbreg_default.predict(X_train.values),
                                               y_train.values))

reg_results_df.loc['xgbreg_default', 'train_rmse'] = xgbreg_default_train_rmse
reg_results_df.loc['xgbreg_default', 'cv_rmse'] = xgbreg_default_cv_rmse
reg_results_df

### Tuning with `hyperopt`

In [ ]:
xgbreg_space = {'max_depth': ho_scope.int(hp.quniform('max_depth', low=1, high=3, q=1)),
                 'n_estimators': ho_scope.int(hp.quniform('n_estimators', low=100, high=500, q=50)),
                'learning_rate': hp.loguniform('learning_rate', low=-4*np.log(10), high=0),
                'gamma': hp.loguniform('gamma', low=-3*np.log(10), high=2*np.log(10)),
                'min_child_weight': ho_scope.int(hp.quniform('min_child_weight', low=1, high=7, q=1)),
                'subsample': hp.uniform('subsample', low=0.25, high=1),
                'colsample_bytree': hp.uniform('colsample_bytree', low=0.25, high=1),
                'colsample_bylevel': hp.uniform('colsample_bylevel', low=0.25, high=1),
                'colsample_bynode': hp.uniform('colsample_bynode', low=0.25, high=1),
                'reg_lambda': hp.loguniform('reg_lambda', low=-2*np.log(10), high=2*np.log(10)),
                'reg_alpha': hp.loguniform('reg_alpha', low=-1*np.log(10), high=1*np.log(10)),
                }

fixed_params = {'objective': 'reg:squarederror',
                'n_jobs': -1,
                'random_state': 27,
               }

def ho_cv_rmse(search_params, X=X, y=y, fixed_params={}, est_name='bayes_ridge'):
    if est_name == 'bayes_ridge':
        est = BayesianRidge(**{**search_params, **fixed_params})
    elif est_name == 'xgb_reg':
        est = XGBRegressor(**{**search_params, **fixed_params})
    return np.sqrt(-np.mean(cross_val_score(estimator=est, 
                                            X=X, 
                                            y=y,
                                            cv=10,
                                            scoring='neg_mean_squared_error',
                                            n_jobs=-1)))

In [ ]:
n = 10
bests = {}

for i in range(n):
    bests[i] = fmin(fn=partial(ho_cv_rmse, X=X_train.values, y=y_train.values, fixed_params=fixed_params, est_name='xgb_reg'),
                    space=xgbreg_space,
                    algo=tpe.suggest,
                    max_evals=10)

In [ ]:
params = {**fixed_params, **bests[4]}
for param in ['max_depth', 'min_child_weight', 'n_estimators']:
    params[param] = int(params[param])
params

xgbreg_ho = XGBRegressor(**params)
xgbreg_ho.fit(X_train.values, y_train.values)

In [ ]:
xgbreg_ho_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=xgbreg_ho, 
                                                  X=X_train.values,
                                                  y=y_train.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1,
                                                 )              
                                 )
                          )

xgbreg_ho.fit(X_train.values, y_train.values)
xgbreg_ho_train_rmse = np.sqrt(mean_squared_error(xgbreg_ho.predict(X_train.values),
                                               y_train.values
                                              )
                            )

In [ ]:
reg_results_df.loc['xgbreg_ho', 'train_rmse'] = xgbreg_ho_train_rmse
reg_results_df.loc['xgbreg_ho', 'cv_rmse'] = xgbreg_ho_cv_rmse
reg_results_df

# Ensembles

## `sklearn` ensemble estimators

In [ ]:
from sklearn.ensemble import (AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor,
                              RandomForestRegressor)

random_state = 27
ensembles = {'ada': AdaBoostRegressor(random_state=random_state), 'bag': BaggingRegressor(),
             'extree': ExtraTreesRegressor(), 'gb': GradientBoostingRegressor(),
             'rf': RandomForestRegressor()}
    
sklearn_ensembles_df = pd.DataFrame(columns=['train_rmse', 'cv_rmse'], index=ensembles.keys())

for model in ensembles:
    ensembles[model].fit(X_train, y_train)
    train_rmse = np.sqrt(mean_squared_error(ensembles[model].predict(X_train), y_train))
    cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=ensembles[model], 
                                                  X=X_train, 
                                                  y=y_train,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))
    sklearn_ensembles_df.loc[model, 'train_rmse'] = train_rmse
    sklearn_ensembles_df.loc[model, 'cv_rmse'] = cv_rmse
    
sklearn_ensembles_df

## Voting

In [ ]:
from sklearn.ensemble import VotingRegressor

voters = []

## Stacking

### Ridge base and xgbreg meta

#### Defaults

In [ ]:
from mlxtend.regressor import StackingCVRegressor

bases = [Ridge() for i in range(5)]
meta = XGBRegressor(**fixed_params)
ridge_base_xgb_meta = StackingCVRegressor(regressors=bases, meta_regressor=meta, random_state=27, n_jobs=-1)

In [ ]:
ridge_base_xgb_meta_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=ridge_base_xgb_meta, 
                                                  X=X_train.values,
                                                  y=y_train.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

ridge_base_xgb_meta.fit(X_train.values, y_train.values)
ridge_base_xgb_meta_train_rmse = np.sqrt(mean_squared_error(ridge_base_xgb_meta.predict(X_train.values),
                                               y_train.values))

reg_results_df.loc['ridge_base_xgb_meta', 'train_rmse'] = ridge_base_xgb_meta_train_rmse
reg_results_df.loc['ridge_base_xgb_meta', 'cv_rmse'] = ridge_base_xgb_meta_cv_rmse
reg_results_df

#### Tuned

In [ ]:
from sklearn.base import clone
bases = [clone(bayes_ridge_ho) for i in range(5)]
meta = xgbreg_ho
tuned_ridge_base_xgb_meta = StackingCVRegressor(regressors=bases, meta_regressor=meta, random_state=27, n_jobs=-1)

In [ ]:
tuned_ridge_base_xgb_meta_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=tuned_ridge_base_xgb_meta, 
                                                  X=X_train.values,
                                                  y=y_train.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

tuned_ridge_base_xgb_meta.fit(X_train.values, y_train.values)
tuned_ridge_base_xgb_meta_train_rmse = np.sqrt(mean_squared_error(tuned_ridge_base_xgb_meta.predict(X_train.values),
                                               y_train.values))

reg_results_df.loc['tuned_ridge_base_xgb_meta', 'train_rmse'] = tuned_ridge_base_xgb_meta_train_rmse
reg_results_df.loc['tuned_ridge_base_xgb_meta', 'cv_rmse'] = tuned_ridge_base_xgb_meta_cv_rmse
reg_results_df

### Stacking tuned ridge meta and xgbreg bases

In [ ]:
bases = [clone(xgbreg_ho) for i in range(5)]
meta = bayes_ridge_ho
tuned_ridge_meta_xgb_base = StackingCVRegressor(regressors=bases, meta_regressor=meta, random_state=27)

In [ ]:
tuned_ridge_meta_xgb_base_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=tuned_ridge_meta_xgb_base, 
                                                  X=X_train.values,
                                                  y=y_train.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

tuned_ridge_meta_xgb_base.fit(X_train.values, y_train.values)
tuned_ridge_meta_xgb_base_train_rmse = np.sqrt(mean_squared_error(tuned_ridge_meta_xgb_base.predict(X_train.values),
                                               y_train.values))

reg_results_df.loc['tuned_ridge_meta_xgb_base', 'train_rmse'] = tuned_ridge_meta_xgb_base_train_rmse
reg_results_df.loc['tuned_ridge_meta_xgb_base', 'cv_rmse'] = tuned_ridge_meta_xgb_base_cv_rmse
reg_results_df

### Mixing ridge and xgb base, with ridge meta

In [ ]:
bases1 = [clone(xgbreg_ho) for i in range(1)]
bases2 = [clone(bayes_ridge_ho) for i in range(1)]
bases = bases1 + bases2

meta = bayes_ridge_ho
tuned_ridge_meta_mixed_base = StackingCVRegressor(regressors=bases, meta_regressor=meta, random_state=27)

In [ ]:
tuned_ridge_meta_mixed_base_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=tuned_ridge_meta_mixed_base, 
                                                  X=X_train.values,
                                                  y=y_train.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

tuned_ridge_meta_mixed_base.fit(X_train.values, y_train.values)
tuned_ridge_meta_mixed_base_train_rmse = np.sqrt(mean_squared_error(tuned_ridge_meta_mixed_base.predict(X_train.values),
                                               y_train.values))

reg_results_df.loc['tuned_ridge_meta_mixed_base', 'train_rmse'] = tuned_ridge_meta_mixed_base_train_rmse
reg_results_df.loc['tuned_ridge_meta_mixed_base', 'cv_rmse'] = tuned_ridge_meta_mixed_base_cv_rmse
reg_results_df

### Mixing ridge and xgb base, with xgb meta

In [ ]:
bases1 = [clone(xgbreg_ho) for i in range(3)]
bases2 = [clone(bayes_ridge_ho) for i in range(3)]
bases = bases1 + bases2

meta = xgbreg_ho
tuned_xgb_meta_mixed_base = StackingCVRegressor(regressors=bases, meta_regressor=meta, random_state=27)

In [ ]:
tuned_xgb_meta_mixed_base_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=tuned_xgb_meta_mixed_base, 
                                                  X=X_train.values,
                                                  y=y_train.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

tuned_xgb_meta_mixed_base.fit(X_train.values, y_train.values)
tuned_xgb_meta_mixed_base_train_rmse = np.sqrt(mean_squared_error(tuned_xgb_meta_mixed_base.predict(X_train.values),
                                               y_train.values))

reg_results_df.loc['tuned_xgb_meta_mixed_base', 'train_rmse'] = tuned_xgb_meta_mixed_base_train_rmse
reg_results_df.loc['tuned_xgb_meta_mixed_base', 'cv_rmse'] = tuned_xgb_meta_mixed_base_cv_rmse
reg_results_df

### Tuning stack with `hyperopt`

#### Mixed base, ridge meta, pretuned base

In [ ]:
def pretuned_base_cv_rmse(bridge_sample, bases=bases, X=X, y=y):
    meta = BayesianRidge(**bridge_sample)
    
    stack_reg = StackingCVRegressor(regressors=bases, meta_regressor=meta, random_state=27)
    loss = np.sqrt(-np.mean(cross_val_score(estimator=stack_reg, 
                                                  X=X.values, 
                                                  y=y.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))
    return {'loss': loss, 'status': STATUS_OK}

In [ ]:
n_runs = 10

bases1 = [clone(xgbreg_ho) for i in range(2)]
bases2 = [clone(bayes_ridge_ho) for i in range(2)]
bases = bases1 + bases2

pretuned_base_bests = {}

for i in range(n_runs):
    trials = Trials()
    pretuned_base_bests[i] = fmin(fn=partial(pretuned_base_cv_rmse, bases=bases, 
                                             X=X_train, y=y_train),
                                  space=bayes_ridge_space,
                                  algo=tpe.suggest,
                                  max_evals=5,
                                  trials=trials)

In [ ]:
pretuned_base_params = pretuned_base_bests[4]
meta = BayesianRidge(**pretuned_base_params)
pretuned_base = StackingCVRegressor(regressors=bases, meta_regressor=meta, random_state=27)

In [ ]:
pretuned_base_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=pretuned_base, 
                                                  X=X_train.values,
                                                  y=y_train.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

pretuned_base.fit(X_train.values, y_train.values)
pretuned_base_train_rmse = np.sqrt(mean_squared_error(pretuned_base.predict(X_train.values),
                                               y_train.values))

reg_results_df.loc['pretuned_base', 'train_rmse'] = pretuned_base_train_rmse
reg_results_df.loc['pretuned_base', 'cv_rmse'] = pretuned_base_cv_rmse
reg_results_df

#### Mixed base, ridge meta, tune full stack at once

In [ ]:
import copy

def stack_hyp_space(num_base=3):
    sample_space = {}
    for i in range(num_base):
        sample_space['xgb' + str(i)] = {'max_depth': ho_scope.int(hp.quniform('xgb_' + str(i) + '-max_depth', low=2, high=6, q=1)),
                     'n_estimators': ho_scope.int(hp.quniform('xgb_' + str(i) + '-n_estimators', low=100, high=500, q=50)),
                     'learning_rate': hp.loguniform('xgb_' + str(i) + '-learning_rate', low=-4*np.log(10), high=0),
                     'gamma': hp.loguniform('xgb_' + str(i) + '-gamma', low=-3*np.log(10), high=2*np.log(10)),
                     'min_child_weight': ho_scope.int(hp.quniform('xgb_' + str(i) + '-min_child_weight', low=2, high=6, q=1)),
                     'subsample': hp.uniform('xgb_' + str(i) + '-subsample', low=0.25, high=1),
                     'colsample_bytree': hp.uniform('xgb_' + str(i) + '-colsample_bytree', low=0.25, high=1),
                     'colsample_bylevel': hp.uniform('xgb_' + str(i) + '-colsample_bylevel', low=0.25, high=1),
                     'colsample_bynode': hp.uniform('xgb_' + str(i) + '-colsample_bynode', low=0.25, high=1),
                     'reg_lambda': hp.loguniform('xgb_' + str(i) + '-reg_lambda', low=-3*np.log(10), high=3*np.log(10)),
                     'reg_alpha': hp.loguniform('xgb_' + str(i) + '-reg_alpha', low=-3*np.log(10), high=3*np.log(10)),
                     }
        sample_space['bridge' + str(i)] = {'alpha_1': hp.loguniform('bridge_' + str(i) + '-alpha_1', low=-9*np.log(10), high=3*np.log(10)),
                     'alpha_2': hp.loguniform('bridge_' + str(i) + '-alpha_2', low=-9*np.log(10), high=3*np.log(10)),
                     'lambda_1': hp.loguniform('bridge_' + str(i) + '-lambda_1', low=-9*np.log(10), high=3*np.log(10)),
                     'lambda_2': hp.loguniform('bridge_' + str(i) + '-lambda_2', low=-9*np.log(10), high=3*np.log(10))}
            
        sample_space['bridge_meta'] = {'alpha_1': hp.loguniform('bridge_meta-alpha_1', low=-9*np.log(10), high=3*np.log(10)),
                     'alpha_2': hp.loguniform('bridge_meta-alpha_2', low=-9*np.log(10), high=3*np.log(10)),
                     'lambda_1': hp.loguniform('bridge_meta-lambda_1', low=-9*np.log(10), high=3*np.log(10)),
                     'lambda_2': hp.loguniform('bridge_meta-lambda_2', low=-9*np.log(10), high=3*np.log(10))}
    
    return sample_space

xgb_fixed_params = {'objective': 'reg:squarederror',
                    'n_jobs': -1,
                     'random_state': 27,
                    }

In [ ]:
def stack_cv_rmse(sample, X=X, y=y, fixed_params={}):
    bases = []
    for name in sample:
        # bases
        if 'xgb' in name and 'meta' not in name:
            params = {**sample[name], **fixed_params['xgb']}
            bases += [XGBRegressor(**params)]
        elif 'bridge' in name and 'meta' not in name:
            params = {**sample[name], **fixed_params['bridge']}
            bases += [BayesianRidge(**params)]
        # meta
        elif 'xgb' in name and 'meta' in name:
            params = {**sample[name], **fixed_params['xgb']}
            meta = XGBRegressor(**params)
        elif 'bridge' in name and 'meta' in name:
            params = {**sample[name], **fixed_params['bridge']}
            meta = BayesianRidge(**params)
    
    stack_reg = StackingCVRegressor(regressors=bases, meta_regressor=meta, random_state=27)
    loss = np.sqrt(-np.mean(cross_val_score(estimator=stack_reg, 
                                                  X=X.values, 
                                                  y=y.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))
    return {'loss': loss, 'status': STATUS_OK}

In [ ]:
n_base = 1
n_runs = 10
space = stack_hyp_space(3)
bests = {}

for i in range(n_runs):
    trials = Trials()
    bests[i] = fmin(fn=partial(stack_cv_rmse, X=X_train, y=y_train, fixed_params=stack_fix_params),
                    space=space,
                    algo=tpe.suggest,
                    max_evals=10,
                    trials=trials)

In [ ]:
bests[3]

In [ ]:
full_stack_tuned_cv_rmse = np.sqrt(-np.mean(cross_val_score(estimator=full_stack_tuned, 
                                                  X=X_train.values,
                                                  y=y_train.values,
                                                  cv=10,
                                                  scoring='neg_mean_squared_error',
                                                  n_jobs=-1)))

full_stack_tuned.fit(X_train.values, y_train.values)
full_stack_tuned_train_rmse = np.sqrt(mean_squared_error(full_stack_tuned.predict(X_train.values),
                                               y_train.values))

reg_results_df.loc['full_stack_tuned', 'train_rmse'] = full_stack_tuned_train_rmse
reg_results_df.loc['full_stack_tuned', 'cv_rmse'] = full_stack_tuned_cv_rmse
reg_results_df

#### Voting meta meta

#### Neural Network meta, mixed base

#### Neural Network meta meta

# Submit

In [ ]:
submit = pd.DataFrame({'Id': X_test.index, 'SalePrice': np.exp(pretuned_base.predict(X_test.values))})

In [ ]:
submit.to_csv('try_again.csv', index=False)
check = pd.read_csv('try_again.csv', index_col=0)
check.head()